In [1]:
%matplotlib inline

from typing import Any

import ipywidgets
import numpy

from matplotlib import pyplot
from matplotlib import ticker
from numpy.typing import NDArray

from reinfocus import vision
from reinfocus.graphics import render
from reinfocus.graphics import world

In [2]:
ObjectSliders = tuple[
    ipywidgets.FloatSlider,
    ipywidgets.FloatSlider,
    ipywidgets.FloatSlider,
    ipywidgets.IntSlider,
    ipywidgets.IntSlider]

In [3]:
def make_object_sliders() -> ObjectSliders:
    distance = ipywidgets.FloatSlider(value=7.5, min=5., max=10., description='distance')
    size = ipywidgets.FloatSlider(value=2., min=0., max=10., description='size')
    r_size = ipywidgets.FloatSlider(value=20., min=0., max=0., description='r_size')
    tfx = ipywidgets.IntSlider(value=16, min=0, max=64, description='tfx')
    tfy = ipywidgets.IntSlider(value=16, min=0, max=64, description='tfy')

    def update_r_sizes(_: dict[str, Any] = {}):
        if size.value == 0.:
            r_size.disabled = False
            r_size.max = 30.
            r_size.value = 20.
        else:
            r_size.disabled = True
            r_size.max = 0.

    size.observe(update_r_sizes, 'value') # type: ignore

    update_r_sizes()

    return distance, size, r_size, tfx, tfy

def flex_layout(flex_type: str) -> ipywidgets.Layout:
    return ipywidgets.Layout(
        flex_flow=f'{flex_type} wrap',
        justify_content='center')

def neat_formatter(x: float, _: int) -> str:
    s = f'{x:g}'
    if 0 < numpy.abs(x) < 1:
        s = s.replace('0', '', 1)
    return s

In [4]:
class WorldViewer(ipywidgets.HBox):
    def __init__(self):
        super().__init__()

        self._world = None

        self._focus_plane = ipywidgets.FloatSlider(
            value=7.5,
            min=5.,
            max=10.,
            description='focus plane')

        self._left_object_sliders = make_object_sliders()
        self._right_object_sliders = make_object_sliders()

        one_controller = ipywidgets.VBox(self._left_object_sliders)

        two_controller = ipywidgets.VBox([
            ipywidgets.VBox([
                ipywidgets.Label(value='left'),
                ipywidgets.VBox(self._left_object_sliders)]),
            ipywidgets.VBox([
                ipywidgets.Label(value='right'),
                ipywidgets.VBox(self._right_object_sliders)])])

        shape_controllers = ipywidgets.Stack(
            [
                one_controller,
                one_controller,
                two_controller,
                two_controller,
                two_controller],
            selected_index=0)

        self._world_choice = ipywidgets.Dropdown(
            options=[
                'one rectangle',
                'one sphere',
                'two rectangles',
                'two spheres',
                'mixed'],
            index=0)

        ipywidgets.jslink(
            (self._world_choice, 'index'),
            (shape_controllers, 'selected_index'))

        self._scan_focus_button = ipywidgets.Button(description='scan focus')
        self._scan_focus_button.on_click(self._on_scan_clicked)

        world_output = ipywidgets.interactive_output(
            self._render_world,
            self._make_render_widget_dict())

        focus_output = ipywidgets.Output()
        self._plot_focus_graph = focus_output.capture(clear_output=True, wait=True)(
            self._plot_focus_graph)

        self.children = [
            ipywidgets.VBox([
                self._world_choice,
                self._focus_plane,
                shape_controllers,
                ipywidgets.HBox(
                    [self._scan_focus_button],
                    layout=flex_layout('row'))]),
            ipywidgets.VBox([world_output], layout=flex_layout('column')),
            ipywidgets.VBox([focus_output], layout=flex_layout('column'))]

        self._plot_focus_graph([], [])

    def _make_render_widget_dict(self) -> dict[str, ipywidgets.Widget]:
        return {
            'focus_plane': self._focus_plane,
            'world_choice': self._world_choice,
            'left_distance': self._left_object_sliders[0],
            'left_size': self._left_object_sliders[1],
            'left_r_size': self._left_object_sliders[2],
            'left_tfx': self._left_object_sliders[3],
            'left_tfy': self._left_object_sliders[4],
            'right_distance': self._right_object_sliders[0],
            'right_size': self._right_object_sliders[1],
            'right_r_size': self._right_object_sliders[2],
            'right_tfx': self._right_object_sliders[3],
            'right_tfy': self._right_object_sliders[4]}

    def _disable_widgets(self, value: bool):
        self._world_choice.disabled = value
        self._focus_plane.disabled = value
        for slider in self._left_object_sliders:
            slider.disabled = value
        for slider in self._right_object_sliders:
            slider.disabled = value
        self._scan_focus_button.disabled = value

    def _on_scan_clicked(self, _: ipywidgets.Button):
        if not isinstance(self._world, world.World):
            return

        self._disable_widgets(True)

        focus_space = numpy.linspace(5., 10., num=100)
        focus_values = []
        for i in focus_space:
            focus_values.append(
                vision.focus_value(render.render(focus_distance=i, world=self._world)))
            self._plot_focus_graph(focus_space[:len(focus_values)], focus_values)

        self._disable_widgets(False)

    def _render_world(
        self,
        focus_plane: float,
        world_choice: str,
        left_distance: float,
        left_size: float,
        left_r_size: float,
        left_tfx: int,
        left_tfy: int,
        right_distance: float,
        right_size: float,
        right_r_size: float,
        right_tfx: int,
        right_tfy: int
    ):
        left = world.ShapeParameters(
            left_distance,
            left_size,
            left_r_size,
            (left_tfx, left_tfy))

        right = world.ShapeParameters(
            right_distance,
            right_size,
            right_r_size,
            (right_tfx, right_tfy))

        world_functions = {
            'one rectangle': lambda: world.one_rect_world(left),
            'one sphere': lambda: world.one_sphere_world(left),
            'two rectangles': lambda: world.two_rect_world(left, right),
            'two spheres': lambda: world.two_sphere_world(left, right),
            'mixed': lambda: world.mixed_world(left, right)}

        self._world = world_functions[world_choice]()

        pyplot.imshow(render.render(focus_distance=focus_plane, world=self._world))
        pyplot.tick_params(bottom=False, labelbottom=False, left=False, labelleft=False)
        ax = pyplot.gca()
        ax.figure.set_figwidth(10.) # type: ignore

    def _plot_focus_graph(self, x: NDArray, y: list[float]):
        ya = numpy.array(y)
        pyplot.plot(x, ya)
        pyplot.grid(axis='x')
        pyplot.xlabel('focus plane')
        pyplot.ylabel('focus value')
        ax = pyplot.gca()
        ax.figure.set_figheight(4.) # type: ignore
        ax.set_xlim(5., 10.)
        ax.set_ylim(0., max([max(ya) * 1.05 if ya.size > 0 else 0, .005]))

        ax.yaxis.set_major_formatter(ticker.FuncFormatter(neat_formatter))
        pyplot.show()


In [5]:
WorldViewer()

WorldViewer(children=(VBox(children=(Dropdown(options=('one rectangle', 'one sphere', 'two rectangles', 'two s…